# AutoGPT with LangChain
## Introduction

After covering BabyAGI library in the earlier lesson, we will delve deep into the mechanics of AutoGPT, another popular type of autonomous agent, demonstrating how it employs multiple tools and memory systems to perform and manage tasks autonomously.

### What is AutoGPT

Unlike traditional language models that generate text based solely on the input they receive, AutoGPT combines a set of tools and a memory system to interact with its environment, retrieve past interactions, and generate more informed responses.

AutoGPT distinguishes itself with three critical features. First, its internet connectivity provides real-time access to information, bolstering its research abilities. Second, its planning capability enables it to generate and pursue sub-tasks that contribute to the completion of a main task. Lastly, it has the power to execute tasks, even to the extent of initiating other AI agents. While these features offer huge potential, they also present challenges, such as inadvertent task looping or prematurely assuming task completion. 

Through internet browser plugins and other forms of access, AutoGPT gathers the information needed to execute tasks. 

An external memory serves as its context-aware module, enabling it to understand its current situation, generate new tasks, and even self-correct if necessary. As a result, AutoGPT operates dynamically, continually re-prioritizing tasks based on the context and situation.

### Tools Setup

We initialize different tools that the AI agent can use to complete tasks. In our case, the tools are Search, WriteFileTool, and ReadFileTool. The Search tool utilizes a GoogleSearchAPIWrapper to fetch real-time information from the internet, which can be employed for questions about current events or queries that need up-to-date information. The WriteFileTool and ReadFileTool manage file-related tasks.

In [1]:
import os

#os.environ["OPENAI_API_KEY"] = "<YOUR-OPENAI-API-KEY>"
#os.environ["GOOGLE_API_KEY"] = "<YOUR-GOOGLE-SEARCH-API-KEY>"
#os.environ["GOOGLE_CSE_ID"] = "<YOUR-CUSTOM-SEARCH-ENGINE-ID>"

#Import the necessary libraries
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

#Set up the tools
search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="Useful for when you need to answer questions about current events. You should ask targeted questions",
        return_direct=True
    ),
    WriteFileTool(),
    ReadFileTool(),
]

Could not import azure.core python package.


### Agent Memory Setup

For the memory, we create the FAISS vector DB (but any other vector DB would work similarly), an efficient similarity search, and clustering of dense vectors. This is paired with an InMemoryDocstore instance for storing documents in memory and an OpenAIEmbeddings model for creating embeddings of the queries. These tools are crucial for the agent's remembering and retrieving past interactions.

AutoGPT has been designed to operate over longer periods. AutoGPT has incorporated a retrieval-based memory system that functions over intermediate agent steps to do that. 

This memory performs a semantic search across embeddings using the vector DB. While such retrieval-based memory is a part of LangChain, it was traditionally used for user and agent interactions, not agent and tools. AutoGPT's new adaptation represents a significant shift in how this memory system is applied.

In [2]:
# Set up the memory
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")
embedding_size = 1536

import faiss
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

### Setting up the Model and AutoGPT

Here we initialize the AutoGPT agent, giving it a name ("Jim") and a role ("Assistant"). We also supplied it with the tools and memory systems that were established in the previous steps. The language model being used here is ChatOpenAI, which is set to have a temperature of 0 (indicating deterministic responses).

In [3]:
# Set up the model and AutoGPT
from langchain.experimental import AutoGPT
from langchain.chat_models import ChatOpenAI

agent = AutoGPT.from_llm_and_tools(
    ai_name="Jim",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    memory=vectorstore.as_retriever()
)

# Set verbose to be true
agent.chain.verbose = True

### Running an Example

Finally, we provided an example task for the AutoGPT agent. This task ("Provide an analysis of the major historical events that led to the French Revolution") is complex and requires the agent to utilize its tools and memory system effectively to generate a response.

The agent takes some minutes to generate the final answer, but we get a peek into all the intermediate computations thanks to having set the verbose variable to True.

Since there are a lot of intermediate computations and the output is very long, we’ll see here only its crucial parts, giving a quick explanation of them.

In [4]:
task = "Provide an analysis of the major historical events that led to the French Revolution"

agent.run([task])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Jim, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Provide an analysis of the major historical events that led to the French Revolution


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: Useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. wr

'I have completed the objective of providing an analysis of the major historical events that led to the French Revolution. I have concluded my tasks.'

As we can see, based on the output Jim, the AI assistant, has performed well, managing to work on three distinct and detailed files. 

Jim’s performance highlights several key capabilities:

- Research and Analysis: The assistant has demonstrated good research and analysis skills. It has managed to delve into the historical context, key events, and the long-term impact of the French Revolution, presenting them in an organized and comprehensible manner.
- Writing and Summarization: The agent displayed good writing skills. It has effectively synthesized complex historical concepts into clear and concise summaries, enabling better understanding for any reader, irrespective of their background knowledge.
- Planning and Workflow Management: It has shown a structured approach in managing the tasks. It completed its research, wrote summaries, and planned for review and presentation, all while maintaining a smooth workflow and ensuring the proper organization and storage of information.
- Autonomy: We are able to see that Jim operated independently, without any user assistance, demonstrating their ability to handle tasks from beginning to end.

The results showcase its capabilities in historical research, analysis, summarization, planning, and information management.